#### Alguien le contó al cliente que el uso de withColumn y withColumnRenamed no correspondian a las best-practices dentro del desarrollo con Apache Spark, por lo tanto ha solicitado que hagan las transformaciones utilizando únicamente instrucciones de tipo Select. Adicional nos ha solicitado agregar algunas operaciones de limpieza no contempladas anteriormente. Presta mucha atención a cada requerimiento solicitado.

#### Por si fuera poco nos ha solicitado filtrar algunos datos que no le serán de utilidad y agregar una columna adicional al dataframe movies_df donde requiere el uso de expresiones regulares (por suerte un especialista nos ha dado las expresiones regulares a utilizar). No estaría mal estudiar un poco sobre expresiones regulares si consideras que no dominas el tema. 

##### Nota: Para poder trabajar con este notebook es necesario haber terminado el ejercicio de la sesión 04

In [1]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
%run utils.py

In [2]:
# NO MODIFICAR CONTENIDO DE ESTA CELDA
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t

# Creación de sesión de Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ejercicio_6") \
    .getOrCreate()

spark.conf.set("spark.sql.session.timeZone", "GMT-6")

# Carga de tablas requeridas
root_path = "../../resources/data/tmp/parquet/"
names_list = ["04/movies", "04/ratings", "04/tags"]
df_dict = read_tmp_df(spark, names_list)

movies_df = df_dict["04/movies"]
ratings_df = df_dict["04/ratings"]
tags_df = df_dict["04/tags"]

movies_df.show(1, False)
ratings_df.show(1)
tags_df.show(1)

+-------+----------------+-------------------------------------------+
|movieId|title           |genres                                     |
+-------+----------------+-------------------------------------------+
|1      |Toy Story (1995)|Adventure|Animation|Children|Comedy|Fantasy|
+-------+----------------+-------------------------------------------+
only showing top 1 row

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      1|   4.0|1225734739|
+------+-------+------+----------+
only showing top 1 row

+------+-------+------+----------+
|userId|movieId|   tag| timestamp|
+------+-------+------+----------+
|224183|    832|acting|1496668827|
+------+-------+------+----------+
only showing top 1 row



#### Actividad 1:
##### TO DO ->    Para el dataframe "movies_df":
- ##### El archivo contiene espacios a la izquiera y derecha en la columna "title". Elimina estos espacios.
    - Apoyate de la funcion trim -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.trim.html#pyspark.sql.functions.trim
- ##### Convierte la columna "genres" en un array, donde cada genero corresponde a una posición del array generado. Como resultado el esquema para la columna "genres" será un ArrayType(StringType()). 
    - Apoyate de la función split de Spark -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.split.html#pyspark.sql.functions.split
- ##### Renombra la columna "movieId" por "movie_id"

##### NO UTILIZAR withColumn NI withColumnRenamed

In [3]:
## TU CODIGO VA EN ESTA CELDA:
# aplicar transformaciones a movies_df
casted_movies_df = movies_df\
    .select(
        f.trim("title").alias("title"),
        f.split(f.col("genres"), "[|]").alias("genres"),
        f.col("movieID").alias("movie_id")
    )

In [4]:
# TEST
# casted_movies_df.show(1, False)

In [5]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
casted_movies_df.show(1, False)
"""
Ejemplo de salida esperada:
+----------------+-------------------------------------------------+--------+
|title           |genres                                           |movie_id|
+----------------+-------------------------------------------------+--------+
|Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1       |
+----------------+-------------------------------------------------+--------+
only showing top 1 row
"""

+----------------+-------------------------------------------------+--------+
|title           |genres                                           |movie_id|
+----------------+-------------------------------------------------+--------+
|Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1       |
+----------------+-------------------------------------------------+--------+
only showing top 1 row



'\nEjemplo de salida esperada:\n+----------------+-------------------------------------------------+--------+\n|title           |genres                                           |movie_id|\n+----------------+-------------------------------------------------+--------+\n|Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1       |\n+----------------+-------------------------------------------------+--------+\nonly showing top 1 row\n'

In [6]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
assert "movie_id" in casted_movies_df.columns
assert "title" in casted_movies_df.columns
assert "genres" in casted_movies_df.columns
assert len(casted_movies_df.columns) == 3

casted_movies_df = casted_movies_df.select("movie_id", "title", "genres")
assert schema_to_ddl(spark, casted_movies_df) == 'movie_id STRING,title STRING,genres ARRAY<STRING>'

#### Actividad 2:
##### TO DO ->    Para el dataframe "ratings_df":
- ##### Renombra la columna "movieId" por "movie_id"
- ##### Renombra la columna "userId" por "user_id"
- ##### Castea la columna "rating" a formato double.
- ##### Convierte la columna "timestamp" a formato TimeStampType con formato logico yyyy-MM-dd HH:mm:ss, la nueva columna generada será "time". 
    - Utiliza la función: timestamp_seconds de Spark -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.timestamp_seconds.html#pyspark.sql.functions.timestamp_seconds
- ##### Elimina la columna "timestamp"
##### NO UTILIZAR withColumn NI withColumnRenamed

In [7]:
## TU CODIGO VA EN ESTA CELDA:
# aplicar transformaciones a ratings_df
casted_ratings_df = ratings_df\
    .select(
        f.col("movieId").alias("movie_id"),
        f.col("userId").alias("user_id"),
        f.col("rating").cast(t.DoubleType()),
        f.timestamp_seconds(f.col("timestamp").cast(t.IntegerType())).alias("time")
    )\
    .drop("timestamp")

In [8]:
# TEST
casted_ratings_df.show(1)
casted_ratings_df.printSchema()

+--------+-------+------+-------------------+
|movie_id|user_id|rating|               time|
+--------+-------+------+-------------------+
|       1|      1|   4.0|2008-11-03 11:52:19|
+--------+-------+------+-------------------+
only showing top 1 row

root
 |-- movie_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- time: timestamp (nullable = true)



In [9]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
casted_ratings_df.show(1)
"""
Ejemplo de salida esperada:
+-------+--------+------+-------------------+
|user_id|movie_id|rating|               time|
+-------+--------+------+-------------------+
|      1|       1|   4.0|2008-11-03 11:52:19|
+-------+--------+------+-------------------+
only showing top 1 row
"""

+--------+-------+------+-------------------+
|movie_id|user_id|rating|               time|
+--------+-------+------+-------------------+
|       1|      1|   4.0|2008-11-03 11:52:19|
+--------+-------+------+-------------------+
only showing top 1 row



'\nEjemplo de salida esperada:\n+-------+--------+------+-------------------+\n|user_id|movie_id|rating|               time|\n+-------+--------+------+-------------------+\n|      1|       1|   4.0|2008-11-03 11:52:19|\n+-------+--------+------+-------------------+\nonly showing top 1 row\n'

In [10]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
assert "user_id" in casted_ratings_df.columns
assert "movie_id" in casted_ratings_df.columns
assert "rating" in casted_ratings_df.columns
assert "time" in casted_ratings_df.columns
assert len(casted_ratings_df.columns) == 4

casted_ratings_df = casted_ratings_df.select("user_id" ,"movie_id", "rating", "time")

assert schema_to_ddl(spark, casted_ratings_df) == 'user_id STRING,movie_id STRING,rating DOUBLE,time TIMESTAMP'

#### Actividad 3:
##### TO DO ->    Para el dataframe "ratings_df":
- ##### Renombra la columna "movieId" por "movie_id"
- ##### Renombra la columna "userId" por "user_id"
- ##### Convierte la columna "timestamp" a formato TimeStampType con formato logico yyyy-MM-dd HH:mm:ss, la nueva columna generada será "time". 
    - ##### Utiliza la función: from_unixtime -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.from_unixtime.html#pyspark.sql.functions.from_unixtime
- ##### Elimina la columna "timestamp"
##### NO UTILIZAR withColumn NI withColumnRenamed

In [11]:
def difference(l1, l2):
    return list(set(l1) - set(l2))

In [12]:
## TU CODIGO VA EN ESTA CELDA:
# aplicar transformaciones a tags_df
casted_tags_df = tags_df\
    .select(
        f.col("userId").alias("user_id"),
        f.col("movieId").alias("movie_id"),
        *difference(tags_df.columns, ["movieId","userId","timestamp"]),
        f.from_unixtime(f.col("timestamp"),"yyyy-MM-dd HH:mm:ss").cast(t.TimestampType()).alias("time")
    )

In [13]:
# TEST
casted_tags_df.show(1)
casted_tags_df.printSchema()

+-------+--------+------+-------------------+
|user_id|movie_id|   tag|               time|
+-------+--------+------+-------------------+
| 224183|     832|acting|2017-06-05 07:20:27|
+-------+--------+------+-------------------+
only showing top 1 row

root
 |-- user_id: string (nullable = true)
 |-- movie_id: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- time: timestamp (nullable = true)



In [14]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
casted_tags_df.show(1)
"""
Ejemplo de salida esperada:
+-------+--------+------+-------------------+
|user_id|movie_id|   tag|               time|
+-------+--------+------+-------------------+
| 224183|     832|acting|2017-06-05 07:20:27|
+-------+--------+------+-------------------+
only showing top 1 row
"""

+-------+--------+------+-------------------+
|user_id|movie_id|   tag|               time|
+-------+--------+------+-------------------+
| 224183|     832|acting|2017-06-05 07:20:27|
+-------+--------+------+-------------------+
only showing top 1 row



'\nEjemplo de salida esperada:\n+-------+--------+------+-------------------+\n|user_id|movie_id|   tag|               time|\n+-------+--------+------+-------------------+\n| 224183|     832|acting|2017-06-05 07:20:27|\n+-------+--------+------+-------------------+\nonly showing top 1 row\n'

In [15]:
casted_tags_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- movie_id: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- time: timestamp (nullable = true)



In [16]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
assert "user_id" in casted_tags_df.columns
assert "movie_id" in casted_tags_df.columns
assert "tag" in casted_tags_df.columns
assert "time" in casted_tags_df.columns
assert len(casted_tags_df.columns) == 4

casted_tags_df = casted_tags_df.select("user_id" ,"movie_id", "tag", "time")

assert schema_to_ddl(spark, casted_tags_df) == 'user_id STRING,movie_id STRING,tag STRING,time TIMESTAMP'

#### Actividad 4:
##### TO DO ->    Para el dataframe resultante de la actividad 1 (casted_movies_df) aplica la siguiente regla de negocio:
- ##### Agrega una columna llamada "year" de tipo IntegerType() de acuerdo a las siguientes condiciones:
    - ##### Si la columna "title" termina con la expresión regular "\\([0-9]{4}\\\)\\$" extraer 4 carácteres desde la posición -5
    - ##### Si la columna "title" termina con la expresión regular '\\([0-9]{4}\\\)"\\$' extraer 4 carácteres desde la posición -6
    - ##### En cualquier otro caso mantener un valor null
    - Utiliza la función substring -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.substring.html#pyspark.sql.functions.substring
    - Adicional necesitarás revisar la siguiente fucion de la clase Column: rlike -> https://spark.apache.org/docs/latest/api/python//reference/pyspark.sql/api/pyspark.sql.Column.rlike.html
        - ##### Para validar qué hace cada expresión regular puedes utilizar https://regex101.com/ o https://regexr.com/ y hacer pruebas con los textos:
            - ##### Toy Story (1995)
            - ##### """Great Performances"" Cats (1998)"
- ##### Filtra todos los registros que no contengan el valor "(no genres listed)" dentro de la columna "genres"
    - Utiliza la función  array_contains -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.array_contains.html#pyspark.sql.functions.array_contains
##### NO UTILIZAR withColumn NI withColumnRenamed

In [17]:
casted_movies_df.show(1, False)

+--------+----------------+-------------------------------------------------+
|movie_id|title           |genres                                           |
+--------+----------------+-------------------------------------------------+
|1       |Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|
+--------+----------------+-------------------------------------------------+
only showing top 1 row



In [18]:
## TU CODIGO VA EN ESTA CELDA:
# aplicar transformaciones a casted_movies_df
transformed_movies_df = casted_movies_df\
    .select(
        *difference(casted_movies_df.columns, ""),
        f.when(f.col("title").rlike("\([0-9]{4}\)$"), f.substring(f.col("title"), -5, 4))
            .when(f.col("title").rlike('\([0-9]{4}\)"$'), f.substring(f.col("title"), -6, 4))
            .otherwise(f.lit("null")).cast(t.IntegerType()).alias("year")
    )\
    .filter(~f.array_contains(f.col("genres"), "(no genres listed)"))

In [19]:
# Test:
transformed_movies_df.show(3, False)
transformed_movies_df.printSchema()

+-----------------------+--------+-------------------------------------------------+----+
|title                  |movie_id|genres                                           |year|
+-----------------------+--------+-------------------------------------------------+----+
|Toy Story (1995)       |1       |[Adventure, Animation, Children, Comedy, Fantasy]|1995|
|Jumanji (1995)         |2       |[Adventure, Children, Fantasy]                   |1995|
|Grumpier Old Men (1995)|3       |[Comedy, Romance]                                |1995|
+-----------------------+--------+-------------------------------------------------+----+
only showing top 3 rows

root
 |-- title: string (nullable = true)
 |-- movie_id: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- year: integer (nullable = true)



In [20]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
transformed_movies_df.show(1)
"""
Ejemplo de salida esperada:
+--------+----------------+--------------------+----+
|movie_id|           title|              genres|year|
+--------+----------------+--------------------+----+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|
+--------+----------------+--------------------+----+
only showing top 1 row
"""

+----------------+--------+--------------------+----+
|           title|movie_id|              genres|year|
+----------------+--------+--------------------+----+
|Toy Story (1995)|       1|[Adventure, Anima...|1995|
+----------------+--------+--------------------+----+
only showing top 1 row



'\nEjemplo de salida esperada:\n+--------+----------------+--------------------+----+\n|movie_id|           title|              genres|year|\n+--------+----------------+--------------------+----+\n|       1|Toy Story (1995)|[Adventure, Anima...|1995|\n+--------+----------------+--------------------+----+\nonly showing top 1 row\n'

In [21]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
assert "movie_id" in transformed_movies_df.columns
assert "title" in transformed_movies_df.columns
assert "genres" in transformed_movies_df.columns
assert "year" in transformed_movies_df.columns
assert len(transformed_movies_df.columns) == 4

transformed_movies_df = transformed_movies_df.select("movie_id" ,"title", "genres", "year")

assert schema_to_ddl(spark, transformed_movies_df) == 'movie_id STRING,title STRING,genres ARRAY<STRING>,year INT'

assert transformed_movies_df.count() == 79477
assert transformed_movies_df.filter(f.col("year").isNull()).count() == 242
assert transformed_movies_df.filter(f.col("year").rlike("^[0-9]{4}$")).count() == 79235

In [22]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(transformed_movies_df, "06/movies"),
       (casted_tags_df, "06/tags"),
       (casted_ratings_df, "06/ratings")]

write_tmp_df(dfs)